In [10]:
import re

In [45]:
data_path = "../data/cleaned_text.txt"

with open(data_path, "r", encoding="utf-8") as f:
    text = f.read()

In [46]:
print(text[0:1351])

التزات النبوي صحيح البخارية للإماراتي عبد الله محمد بن اسماعيل البخاري الجيفي رحمه الله تعالى ضبطه ورقمه وذكر تكرار مواضعه وشرح الفاظه وحمله وخرج أحاديثه في صحيح مسلم ووضع فهارسة الدكتور مصطفى ديب البغا دكتوراه في الشربية الإسلامية من جامعة الأزهر مدرس في كلية الشريعة جامعة وشر دمشق بيروت للطباعة والنشر والتوزيع صحيح البخاري الجزء الأول بدء الوحي قال الشيخ الإمام الحافظ أبو عبد الله محمد بن إسمعيل بن إبراهيم بن المغيرة البخاري رحمه الله تعالى آمين باب كيف كان بدء الوحي إلى رسول الله الله وقول الله جل ذكره إنا أوحينا إليك كما أوحينا إلى نوح والنبيين من بعده /النساء / ۱ حدثنا الحميدي عبد الله بن الزبير قال حدثنا سفيان قال حدثنا يحى بن سعيد الأنصاري قال أخبرني محمد بن إبراهيم التيمي أنه سمع علقمة بن وقاص الليتي يقول سمعت عمر بن الخطاب رضي الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول (إنما الأعمال بالنيات وإنما لكل امرئ ما نوى فمن كانت هجرته إلى دنيا يصيبها أو إلى أمرأة ينكحها فهجرته إلى ما هاجر إليه) حدثنا يحيى بن بكير قال حدثنا الليث عن عقيل عن ابن شهاب عن عروة ابن الزب

In [7]:
sub_text = text[0:1351]

# Unigram tokenizer

For the tokenization algorithm, we selected Unigram language model instead of BPE or WordPiece. The Unigram model is particularly effective for morphologically rich languages like Arabic because it generates a probabilistic set of subword candidates and then prunes them based on likelihood. This allows the tokenizer to capture meaningful subwords (roots, prefixes, suffixes) more flexibly compared to deterministic methods like BPE, which always merges the most frequent pairs.

In [56]:
import sentencepiece as spm
import os


In [67]:
import os
import sentencepiece as spm


data_path = os.path.join("..", "data", "cleaned_text.txt")
tokenizer_path = os.path.join("..", "tokenizer_model", "hadith_unigram")

# Check input file
assert os.path.exists(data_path), f"File not found: {data_path}"
assert os.path.getsize(data_path) > 0, "Input file is empty!"

# Ensure output directory exists
os.makedirs(os.path.dirname(tokenizer_path), exist_ok=True)

# Train tokenizer
spm.SentencePieceTrainer.Train(
    input=str(data_path),
    model_prefix=str(tokenizer_path),
    vocab_size=22000,
    model_type='unigram',
    character_coverage=0.9995,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)


In [69]:
sp = spm.SentencePieceProcessor()
sp.load(tokenizer_path + ".model" )

True

In [73]:
# Example text
text = """
السلام عليكم ورحمه الله وبركاته ممكن تيجي تتغدي عندي النهارده  845 786
sdfsdg
"""

# Encode to pieces (subwords)
print("Pieces:", sp.encode_as_pieces(text))

# Encode to IDs
print("IDs:", sp.encode_as_ids(text))

# Decode back
print("Decoded:", sp.decode_ids(sp.encode_as_ids(text)))

Pieces: ['▁السلام', '▁عليكم', '▁و', 'رحمه', '▁الله', '▁وبركاته', '▁م', 'مكن', '▁ت', 'يج', 'ي', '▁ت', 'ت', 'غ', 'دي', '▁عندي', '▁النهار', 'ده', '▁', '845', '▁', '786', '▁', 'sdfsdg']
IDs: [222, 331, 16, 9154, 5, 6490, 116, 8150, 129, 3447, 32, 129, 31, 3774, 1669, 794, 732, 1538, 4, 1, 4, 1, 4, 1]
Decoded: السلام عليكم ورحمه الله وبركاته ممكن تيجي تتغدي عندي النهارده  ⁇   ⁇   ⁇ 
